In [132]:
import pandas as pd
import numpy as np
import pymongo
import itertools
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["flick"]

### Initialize

In [123]:
mycol = mydb["genres_data"]
mycol.drop()
genres_df=pd.read_csv('genres.csv')
genre_json=genres_df.to_json(orient='records')
genre_json=eval(genre_json)
genre_data=genre_json
x = mycol.insert_many(genre_data)

mycol = mydb["movies_data"]
mycol.drop()
movies_df=pd.read_csv('movies.csv')
movies_json=movies_df.to_json(orient='records')
movies_json=eval(movies_json)
movies_data=movies_json
movies_data[0]
x = mycol.insert_many(movies_data)

### New review

In [125]:
#inserting new review
uid=1
movieId=42
rating=4
mycol = mydb["reviews"]
record={ "uid": uid, "movieId": movieId,"rating":rating }
mycol.insert_one(record)
#generating userInput table
userInput=[]
for x in mycol.find({"uid":uid},{ "_id": 0}):
  userInput.append(x)
userInput=pd.DataFrame(userInput)
userInput=userInput.drop(['uid'], axis = 1) 
#fetching genres data from database
mycol = mydb["genres_data"]
data=[]
for x in mycol.find({},{"_id":0}):
    data.append(x)
genres_df=pd.DataFrame(data)
#generating userProfile
userGenre=genres_df[genres_df['movieId'].isin(userInput['movieId'].tolist())]
userGenre.drop('movieId',1,inplace=True)
userGenre.reset_index(drop=True)
userProfile = userGenre.transpose().dot(userInput.rating.values)
#getting new recommendations
genreTable=genres_df.copy()
genreTable.set_index('movieId',inplace=True)
recommend_df=((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommend_df.sort_values(ascending=False,inplace=True)
mycol = mydb["user_recommendation_data"]
recommendation_json=eval(recommend_df.to_json())
recommendation_data={"uid":1,"recommendation_data":recommendation_json}
mycol.delete_one({"uid":uid})
x = mycol.insert_one(recommendation_data)
recommend_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


movieId
27344     0.863636
71999     0.818182
49593     0.818182
26093     0.795455
141982    0.772727
dtype: float64

###    Recommendation

In [160]:
mycol = mydb["user_recommendation_data"]
x=mycol.find_one({"uid":1},{"_id":0})
data=x['recommendation_data']
# recommend_df=pd.Series(data)
# recommend_df.head()
data=dict(itertools.islice(data.items(),5))
movies_id=list(data.keys())

mycol=mydb['movies_data']
movies_data=[]
for i in mycol.find({},{"_id":0}):
    if(str(i["movieId"]) in movies_id):
        movies_data.append(i)
    #print(i["movieId"])
movies_data

[{'movieId': 26093,
  'title': 'Wonderful World of the Brothers Grimm, The (1962)',
  'genres': 'Adventure|Animation|Children|Comedy|Drama|Fantasy|Musical|Romance'},
 {'movieId': 27344,
  'title': 'Revolutionary Girl Utena: Adolescence of Utena (a.k.a. Revolutionary Girl Utena the Movie) (Shoujo kakumei Utena: Adolescence mokushiroku) (1999)',
  'genres': 'Action|Adventure|Animation|Comedy|Drama|Fantasy|Romance'},
 {'movieId': 49593,
  'title': 'She (1965)',
  'genres': 'Action|Adventure|Drama|Fantasy|Horror|Romance|Sci-Fi'},
 {'movieId': 71999,
  'title': 'Aelita: The Queen of Mars (Aelita) (1924)',
  'genres': 'Action|Adventure|Drama|Fantasy|Romance|Sci-Fi|Thriller'},
 {'movieId': 141982,
  'title': 'Mrs Miracle (2009)',
  'genres': 'Children|Comedy|Drama|Fantasy|Romance|Sci-Fi'}]